In [6]:
# -*- coding: utf-8 -*-
"""UAS_Penalaran_Komputer_Tahap_5.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1C_YOUR_NOTEBOOK_ID_HERE
"""

# Instalasi Library
print("Menginstal library yang dibutuhkan untuk Tahap 5...")
!pip install pandas scikit-learn numpy joblib transformers sentence-transformers > /dev/null 2>&1
print("Instalasi library selesai.")

# Koneksi Google Drive
print("Menghubungkan Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive terhubung.")

Menginstal library yang dibutuhkan untuk Tahap 5...
Instalasi library selesai.
Menghubungkan Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive terhubung.


In [7]:
import pandas as pd
import os
import joblib # Untuk memuat model TF-IDF
import numpy as np # Untuk memuat BERT embeddings
from sentence_transformers import SentenceTransformer # Untuk model embedding BERT
from sklearn.feature_extraction.text import TfidfVectorizer # Untuk vectorizer TF-IDF
from sklearn.metrics.pairwise import cosine_similarity
import re # Untuk clean_text
import json # Untuk memuat queries.json
from typing import List, Dict, Any, Union, Tuple

# --- Fungsi untuk membuat jalur (diulang agar tersedia di Tahap 5) ---
def create_path(folder_name: str) -> str:
    path = os.path.join('/content/drive/MyDrive/', folder_name)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Folder '{path}' berhasil dibuat di Google Drive.")
    else:
        print(f"Folder '{path}' sudah ada di Google Drive.")
    return path

# --- Fungsi clean_text (diulang agar tersedia di Tahap 5) ---
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = re.sub(r'M a h ka m a h A g u n g R e p u blik In d o n esia\n', '', text)
    text = re.sub(r'Disclaimer\n', '', text)
    text = re.sub(r'Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n', '', text)
    text = re.sub(r'pelaksanaan fungsi peradilan\. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu\.\n', '', text)
    text = re.sub(r'Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n', '', text)
    text = re.sub(r'Email : kepaniteraan@mahkamahagung\.go\.id\s+Telp : 021-384 3348 \(ext\.318\)\n', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- Fungsi untuk memuat cases.csv (diulang agar tersedia di Tahap 5) ---
def load_cases_data(processed_data_folder: str) -> pd.DataFrame:
    cases_csv_path = os.path.join(processed_data_folder, 'cases.csv')
    print(f"Mencoba memuat cases.csv dari: {cases_csv_path}")
    if not os.path.exists(cases_csv_path):
        print(f"ERROR: File cases.csv TIDAK DITEMUKAN di '{cases_csv_path}'. Pastikan Tahap 2 sudah selesai dengan benar.")
        return pd.DataFrame()
    try:
        df = pd.read_csv(cases_csv_path)
        if df.empty:
            print(f"Peringatan: File cases.csv ditemukan tetapi KOSONG.")
        else:
            print(f"Berhasil memuat {len(df)} kasus dari cases.csv.")
            for col in ['judul', 'ringkasan_fakta', 'argumen_hukum_utama', 'text_pdf', 'amar', 'amar_lainnya', 'catatan_amar']:
                if col in df.columns:
                    df[col] = df[col].astype(str).fillna('')
        return df
    except Exception as e:
        print(f"ERROR: Gagal memuat cases.csv: {e}")
        return pd.DataFrame()

# --- Fungsi ekstraksi solusi (diulang agar tersedia di Tahap 5) ---
def extract_solution_text(case_row: pd.Series) -> str:
    if 'amar' in case_row and case_row['amar'] and case_row['amar'].strip() != 'nan':
        return f"Amar Putusan: {case_row['amar'].strip()}"
    elif 'catatan_amar' in case_row and case_row['catatan_amar'] and case_row['catatan_amar'].strip() != 'nan':
        return f"Catatan Amar: {case_row['catatan_amar'].strip()}"
    elif 'ringkasan_fakta' in case_row and case_row['ringkasan_fakta'] and case_row['ringkasan_fakta'].strip() != 'nan':
        return f"Ringkasan Fakta (Kemungkinan Dakwaan): {case_row['ringkasan_fakta'].strip()}"
    elif 'argumen_hukum_utama' in case_row and case_row['argumen_hukum_utama'] and case_row['argumen_hukum_utama'].strip() != 'nan':
        return f"Argumen Hukum Utama: {case_row['argumen_hukum_utama'].strip()}"
    return "Solusi tidak dapat diekstrak dari kasus ini."


# --- Load Model dan Vektor dari Tahap 3 ---
# Konfigurasi Jalur
base_drive_path = '/content/drive/MyDrive/CBR_Data'
models_output_folder = os.path.join(base_drive_path, 'models')
processed_data_folder = os.path.join(base_drive_path, 'data/processed') # Untuk load cases.csv
eval_data_folder = create_path(os.path.join(base_drive_path, 'data/eval')) # Untuk menyimpan hasil evaluasi

# Inisialisasi variabel global untuk Tahap 5
loaded_vectorizer = None # TfidfVectorizer jika TF-IDF
loaded_embedding_model = None # SentenceTransformer jika BERT
loaded_case_vectors = None # Numpy array atau sparse matrix dari kasus
loaded_vectorizer_type = None # "TF-IDF" atau "BERT_Embedding"
df_cases = pd.DataFrame() # Inisialisasi df_cases

print("Memuat model dan vektor dari Tahap 3...")
try:
    with open(os.path.join(models_output_folder, 'vectorizer_type.txt'), 'r') as f:
        loaded_vectorizer_type = f.read().strip()
    print(f"Tipe vectorizer yang digunakan: {loaded_vectorizer_type}")

    if loaded_vectorizer_type == "TF-IDF":
        loaded_vectorizer = joblib.load(os.path.join(models_output_folder, 'tfidf_vectorizer.joblib'))
        loaded_case_vectors = joblib.load(os.path.join(models_output_folder, 'tfidf_case_vectors.joblib'))
        print("TF-IDF vectorizer dan case vectors berhasil dimuat.")
    elif loaded_vectorizer_type == "BERT_Embedding":
        print("Memuat ulang SentenceTransformer model untuk BERT Embedding...")
        loaded_embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        loaded_case_vectors = np.load(os.path.join(models_output_folder, 'bert_case_vectors.npy'))
        print("BERT embedding model dan case vectors berhasil dimuat.")
    else:
        print("ERROR: Tipe vectorizer tidak dikenal.")

except FileNotFoundError:
    print("ERROR: File model Tahap 3 tidak ditemukan. Pastikan Tahap 3 sudah selesai dan menyimpan outputnya.")
except Exception as e:
    print(f"ERROR: Gagal memuat model/vektor dari Tahap 3: {e}")

# Muat df_cases untuk Tahap 5
df_cases = load_cases_data(processed_data_folder)

# --- Definisi Ulang Fungsi retrieve dari Tahap 3/4 ---
# Menggunakan variabel global yang baru saja dimuat.
def retrieve(query: str, k: int = 5) -> List[Dict[str, Any]]:
    print(f"\nMelakukan retrieval untuk query: '{query[:50]}...' (top-k={k})")

    processed_query = clean_text(query)
    query_vector = None

    if loaded_vectorizer_type == "TF-IDF":
        if loaded_vectorizer is not None:
            query_vector = loaded_vectorizer.transform([processed_query])
        else:
            print("ERROR: TF-IDF Vectorizer tidak dimuat.")
            return []
    elif loaded_vectorizer_type == "BERT_Embedding":
        if loaded_embedding_model is not None:
            query_vector = loaded_embedding_model.encode([processed_query], convert_to_tensor=True).cpu().numpy()
        else:
            print("ERROR: BERT Embedding Model tidak dimuat.")
            return []
    else:
        print("ERROR: Tipe vectorizer tidak dikenal atau belum dimuat.")
        return []

    if query_vector is None or loaded_case_vectors is None:
        print("ERROR: Query vector atau loaded_case_vectors kosong. Tidak bisa menghitung kemiripan.")
        return []

    try:
        import scipy.sparse # Import lokal di sini
    except ImportError:
        pass

    is_sparse_matrix = False
    try:
        if 'scipy.sparse' in globals() and hasattr(scipy.sparse, 'csr_matrix'):
            is_sparse_matrix = isinstance(loaded_case_vectors, scipy.sparse.csr_matrix)
    except NameError:
        pass

    if not isinstance(loaded_case_vectors, np.ndarray) and not is_sparse_matrix:
         print(f"ERROR: loaded_case_vectors memiliki tipe yang tidak didukung untuk similarity: {type(loaded_case_vectors)}")
         return []

    try:
        similarities = cosine_similarity(query_vector, loaded_case_vectors).flatten()
    except Exception as e:
        print(f"ERROR: Gagal menghitung cosine similarity: {e}")
        print(f"  Query vector shape: {query_vector.shape if query_vector is not None else 'None'}")
        print(f"  Corpus vectors shape: {loaded_case_vectors.shape if loaded_case_vectors is not None else 'None'}")
        return []

    if len(similarities) < k:
        print(f"Peringatan: Jumlah sampel ({len(similarities)}) kurang dari k ({k}). Mengembalikan semua sampel.")
        top_k_indices = np.arange(len(similarities))
    else:
        top_k_indices = np.argpartition(similarities, -k)[-k:]

    sorted_top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])][::-1]

    results = []
    original_valid_indices_mask = df_cases['text_pdf'].apply(lambda x: isinstance(x, str) and x.strip() != '')
    original_valid_indices = df_cases[original_valid_indices_mask].index.tolist()

    if len(original_valid_indices) != loaded_case_vectors.shape[0]:
        print(f"ERROR: Ketidakcocokan panjang antara original_valid_indices ({len(original_valid_indices)}) dan loaded_case_vectors ({loaded_case_vectors.shape[0]}).")
        print("Ini bisa terjadi jika ada dokumen yang dibuang setelah embedding atau kesalahan indexing.")
        return []

    for idx_in_corpus in sorted_top_k_indices:
        original_df_index = original_valid_indices[idx_in_corpus]
        case_info = df_cases.loc[original_df_index]
        similarity_score = similarities[idx_in_corpus]
        results.append({
            "case_id": case_info['nomor'],
            "judul": case_info['judul'],
            "klasifikasi": case_info['klasifikasi'],
            "similarity_score": float(similarity_score),
            "link": case_info['link']
        })

    print(f"Retrieval selesai. Ditemukan {len(results)} kasus.")
    return results

# --- Definisi Ulang Fungsi predict_outcome dari Tahap 4 ---
# Menggunakan variabel global yang baru saja dimuat.
def predict_outcome(query: str, k: int = 5) -> Dict[str, Any]:
    print(f"\nMemprediksi solusi untuk query: '{query[:50]}...'")

    top_k_cases_retrieved = retrieve(query, k=k)

    if not top_k_cases_retrieved:
        print("Tidak ada kasus terkemuka yang ditemukan. Tidak dapat memprediksi solusi.")
        return {
            "predicted_solution": "Tidak dapat memprediksi solusi karena tidak ada kasus serupa ditemukan.",
            "top_k_case_ids": [],
            "top_k_details": []
        }

    solutions_with_similarity = []
    top_k_case_ids = []
    top_k_details = []

    for case_data in top_k_cases_retrieved:
        case_id = case_data['case_id']
        similarity_score = case_data['similarity_score']
        original_link = case_data['link']

        found_case_row = df_cases[df_cases['nomor'] == case_id]
        if found_case_row.empty:
             found_case_row = df_cases[df_cases['link'] == original_link]

        if not found_case_row.empty:
            solution_text = extract_solution_text(found_case_row.iloc[0])
            solutions_with_similarity.append({
                "solution": solution_text,
                "similarity": similarity_score
            })
            top_k_case_ids.append(case_id)
            top_k_details.append({
                "case_id": case_id,
                "judul": case_data['judul'],
                "klasifikasi": case_data['klasifikasi'],
                "similarity_score": similarity_score,
                "solution_text_preview": solution_text[:100] + "..." if len(solution_text) > 100 else solution_text
            })
        else:
            print(f"Peringatan: Detail kasus lengkap untuk case_id {case_id} tidak ditemukan di df_cases.")
            solutions_with_similarity.append({
                "solution": "Detail kasus tidak ditemukan.",
                "similarity": similarity_score
            })
            top_k_case_ids.append(case_id) # Pastikan hanya case_id, bukan dict
            top_k_details.append({
                "case_id": case_id,
                "judul": case_data['judul'],
                "klasifikasi": case_data['klasifikasi'],
                "similarity_score": similarity_score,
                "solution_text_preview": "Detail kasus tidak ditemukan."
            })


    predicted_solution = "Tidak dapat memprediksi solusi."
    if solutions_with_similarity:
        sorted_solutions = sorted(solutions_with_similarity, key=lambda x: x['similarity'], reverse=True)
        predicted_solution = sorted_solutions[0]['solution']
        print(f"DEBUG: Solusi prediksi diambil dari kasus dengan skor kemiripan tertinggi ({sorted_solutions[0]['similarity']:.4f}).")

    print("Prediksi solusi selesai.")
    return {
        "predicted_solution": predicted_solution,
        "top_k_case_ids": top_k_case_ids,
        "top_k_details": top_k_details
    }

print("Semua fungsi utilitas dan model Tahap 5 berhasil didefinisikan.")

Folder '/content/drive/MyDrive/CBR_Data/data/eval' sudah ada di Google Drive.
Memuat model dan vektor dari Tahap 3...
Tipe vectorizer yang digunakan: BERT_Embedding
Memuat ulang SentenceTransformer model untuk BERT Embedding...
BERT embedding model dan case vectors berhasil dimuat.
Mencoba memuat cases.csv dari: /content/drive/MyDrive/CBR_Data/data/processed/cases.csv
Berhasil memuat 65 kasus dari cases.csv.
Semua fungsi utilitas dan model Tahap 5 berhasil didefinisikan.


In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import json # Untuk memuat queries.json

def eval_retrieval(queries_results: List[Dict[str, Any]], k: int = 5) -> pd.DataFrame:
    """
    Mengevaluasi performa retrieval (Precision@k, Recall@k, F1-score).
    Args:
        queries_results (List[Dict[str, Any]]): Daftar hasil query, masing-masing dengan 'query_id', 'query_text', 'ground_truth_case_id', 'retrieved_cases'.
                                                Ini adalah format output saat mengumpulkan retrieval_test_results_for_eval di Sel 4.
        k (int): Jumlah top-k item yang akan dievaluasi.
    Returns:
        pd.DataFrame: DataFrame berisi metrik evaluasi per query.
    """
    print(f"\n--- Memulai Evaluasi Retrieval (k={k}) ---")
    metrics_list = [] # <<< PERBAIKAN DI SINI: Pastikan ini adalah 'metrics_list'

    # Pastikan data dan model yang dibutuhkan tersedia
    if df_cases.empty or loaded_case_vectors is None or df_cases['text_pdf'].count() == 0:
        print("ERROR: df_cases kosong atau vektor kasus tidak dimuat/kosong. Tidak dapat melakukan evaluasi retrieval.")
        return pd.DataFrame()

    for query_data in queries_results: # Menggunakan queries_results yang sudah berisi retrieved_cases
        query_id = query_data['query_id']
        query_text = query_data['query_text']
        ground_truth_ids = set(query_data.get('ground_truth_case_id', [])) # Ground truth bisa kosong
        retrieved_cases = query_data.get('retrieved_cases', []) # Ambil hasil retrieve yang sudah ada

        print(f"  Mengevaluasi Query ID: {query_id}")

        retrieved_ids = {case['case_id'] for case in retrieved_cases}

        # Hitung True Positives (TP) - item yang relevan DAN berhasil diretriev
        true_positives = len(ground_truth_ids.intersection(retrieved_ids))

        # Hitung False Positives (FP) - item yang diretriev tapi tidak relevan
        false_positives = len(retrieved_ids) - true_positives

        # Hitung False Negatives (FN) - item yang relevan tapi TIDAK berhasil diretriev
        false_negatives = len(ground_truth_ids) - true_positives

        # Presisi@k: Berapa persen dari yang diretriev adalah relevan
        precision_at_k = true_positives / k if k > 0 else 0.0

        # Recall@k: Berapa persen dari yang relevan berhasil diretriev
        recall_at_k = true_positives / len(ground_truth_ids) if len(ground_truth_ids) > 0 else 0.0

        # F1-score: Harmonic mean dari precision dan recall
        f1_at_k = (2 * precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) > 0 else 0.0

        metrics_list.append({
            "query_id": query_id,
            "precision_at_k": precision_at_k,
            "recall_at_k": recall_at_k,
            "f1_score_at_k": f1_at_k,
            "true_positives": true_positives,
            "false_positives": false_positives,
            "false_negatives": false_negatives,
            "num_retrieved": len(retrieved_ids),
            "num_ground_truth": len(ground_truth_ids)
        })

    if not metrics_list: # Pemeriksaan ini menjadi aman setelah inisialisasi
        print("Peringatan: Tidak ada hasil evaluasi yang dikumpulkan. Mengembalikan DataFrame kosong.")
        return pd.DataFrame()

    df_metrics = pd.DataFrame(metrics_list)
    print("\nEvaluasi Retrieval Selesai.")
    return df_metrics

# --- Fungsi opsional untuk evaluasi prediksi (lebih kompleks, jika target prediksi adalah label diskrit) ---
def eval_prediction(predictions: List[Dict[str, Any]], ground_truth_solutions: Dict[str, str] = None) -> pd.DataFrame:
    """
    (Opsional/Lanjutan) Mengevaluasi performa prediksi solusi.
    Membutuhkan ground truth solusi yang eksplisit untuk setiap query,
    yang seringkali sulit didapatkan untuk teks bebas.
    """
    print("\n--- Memulai Evaluasi Prediksi (Analisis Kualitatif/Sederhana) ---")
    print("Implementasi ini memerlukan ground truth solusi yang terstruktur untuk setiap query agar metrik kuantitatif bisa dihitung.")
    print("Untuk saat ini, akan membuat tabel hasil prediksi untuk analisis manual/kualitatif.")

    prediction_metrics_list = []
    for pred in predictions:
        is_predicted_successfully = "True" if pred['predicted_solution'].strip() != "Tidak dapat memprediksi solusi karena tidak ada kasus serupa ditemukan." and pred['predicted_solution'].strip() != "" else "False"
        prediction_metrics_list.append({
            "query_id": pred['query_id'],
            "query_text_preview": pred['query_text'][:100] + "..." if len(pred['query_text']) > 100 else pred['query_text'],
            "predicted_solution_status": is_predicted_successfully,
            "predicted_solution_preview": pred['predicted_solution'][:200] + "..." if len(pred['predicted_solution']) > 200 else pred['predicted_solution'],
            "top_5_case_ids": pred['top_5_case_ids'],
            "top_5_details_json": pred['top_5_details_json'] # Simpan juga detail lengkap
        })

    df_prediction_metrics = pd.DataFrame(prediction_metrics_list)
    print("Evaluasi Prediksi Selesai (Membuat Dataframe untuk Analisis Kualitatif).")
    return df_prediction_metrics

print("Fungsi evaluasi retrieval dan prediksi berhasil didefinisikan.")

Fungsi evaluasi retrieval dan prediksi berhasil didefinisikan.


In [13]:
import json # Pastikan json diimpor di sel ini juga

# --- Output Paths ---
base_drive_path = '/content/drive/MyDrive/CBR_Data'
eval_data_folder = create_path(os.path.join(base_drive_path, 'data/eval'))
results_folder = create_path(os.path.join(base_drive_path, 'data/results')) # Folder hasil Tahap 4

retrieval_metrics_csv_path = os.path.join(eval_data_folder, 'retrieval_metrics.csv')
prediction_metrics_csv_path = os.path.join(eval_data_folder, 'prediction_metrics.csv') # Sesuai tugas

# --- Load queries.json dari Tahap 3 ---
# 'sample_queries_from_json' berisi data awal dari queries.json (query_id, text, ground_truth_case_id)
sample_queries_from_json = []
try:
    if os.path.exists(os.path.join(eval_data_folder, 'queries.json')):
        with open(os.path.join(eval_data_folder, 'queries.json'), 'r', encoding='utf-8') as f:
            sample_queries_from_json = json.load(f)
        print(f"Berhasil memuat {len(sample_queries_from_json)} query dari queries.json.")
    else:
        print(f"ERROR: File queries.json TIDAK DITEMUKAN di '{os.path.join(eval_data_folder, 'queries.json')}'. Pastikan Tahap 3 sudah menyimpan query uji.")
except Exception as e:
    print(f"ERROR: Gagal memuat queries.json: {e}")

# --- Siapkan data untuk evaluasi retrieval: Jalankan retrieval untuk setiap query.json ---
retrieval_test_results_for_eval = [] # Akan diisi dengan data yang diperlukan eval_retrieval
prediction_results_for_csv_output = [] # Akan diisi dengan data untuk predictions.csv

if (not df_cases.empty and loaded_case_vectors is not None and df_cases['text_pdf'].count() > 0 and
    'retrieve' in globals() and callable(retrieve) and
    'predict_outcome' in globals() and callable(predict_outcome) and
    sample_queries_from_json):

    print("\n--- Melakukan Retrieval dan Prediksi untuk Mengumpulkan Hasil Evaluasi ---")
    for query_data in sample_queries_from_json: # Iterasi dari queries.json
        query_id = query_data['query_id']
        query_text = query_data['text'] # Ambil 'text' dari queries.json
        ground_truth_ids = query_data.get('ground_truth_case_id', [])

        print(f"  Memproses Query ID: {query_id}")

        # 1. Lakukan Retrieval dan simpan untuk evaluasi retrieval
        try:
            retrieved_cases = retrieve(query_text, k=5)
            retrieval_test_results_for_eval.append({
                "query_id": query_id,
                "query_text": query_text, # Simpan 'query_text' di sini
                "ground_truth_case_id": ground_truth_ids,
                "retrieved_cases": retrieved_cases
            })
        except Exception as e:
            print(f"  ERROR: Gagal melakukan retrieval untuk query {query_id} (saat evaluasi): {e}")
            retrieval_test_results_for_eval.append({
                "query_id": query_id, "query_text": query_text, "ground_truth_case_id": ground_truth_ids,
                "retrieved_cases": [], "retrieval_error": str(e)
            })

        # 2. Lakukan Prediksi dan simpan untuk predictions.csv
        try:
            predicted_output = predict_outcome(query_text, k=5)
            prediction_results_for_csv_output.append({
                "query_id": query_id,
                "query_text": query_text, # Simpan query text lengkap
                "predicted_solution": predicted_output['predicted_solution'],
                "top_5_case_ids": ", ".join(predicted_output['top_k_case_ids']),
                "top_5_details_json": json.dumps(predicted_output['top_k_details'], ensure_ascii=False)
            })
        except Exception as e:
            print(f"  ERROR: Gagal melakukan prediksi untuk query {query_id} (saat evaluasi): {e}")
            prediction_results_for_csv_output.append({
                "query_id": query_id, "query_text": query_text,
                "predicted_solution": "ERROR: " + str(e),
                "top_5_case_ids": "", "top_5_details_json": ""
            })

    # --- Evaluasi Retrieval dan Simpan Metrik ---
    df_retrieval_metrics = pd.DataFrame()
    if retrieval_test_results_for_eval:
        df_retrieval_metrics = eval_retrieval(retrieval_test_results_for_eval, k=5)
        if not df_retrieval_metrics.empty:
            avg_precision = df_retrieval_metrics['precision_at_k'].mean()
            avg_recall = df_retrieval_metrics['recall_at_k'].mean()
            avg_f1 = df_retrieval_metrics['f1_score_at_k'].mean()

            print("\n--- Ringkasan Metrik Retrieval ---")
            print(f"Precision@5 Rata-rata: {avg_precision:.4f}")
            print(f"Recall@5 Rata-rata:    {avg_recall:.4f}")
            print(f"F1-Score@5 Rata-rata:  {avg_f1:.4f}")

            try:
                df_retrieval_metrics.to_csv(retrieval_metrics_csv_path, index=False, encoding='utf-8')
                print(f"\nMetrik retrieval berhasil disimpan ke: {retrieval_metrics_csv_path}")
            except Exception as e:
                print(f"ERROR: Gagal menyimpan retrieval_metrics.csv: {e}")
        else:
            print("DataFrame metrik retrieval kosong. Tidak ada hasil evaluasi untuk disimpan.")
    else:
        print("Tidak ada hasil retrieval untuk dievaluasi.")


    # --- Simpan Hasil Prediksi untuk Analisis Kualitatif ---
    df_prediction_metrics = pd.DataFrame()
    if prediction_results_for_csv_output:
        # Panggil eval_prediction dengan data yang sudah terkumpul
        df_prediction_metrics = eval_prediction(prediction_results_for_csv_output)
        if not df_prediction_metrics.empty:
            try:
                df_prediction_metrics.to_csv(prediction_metrics_csv_path, index=False, encoding='utf-8')
                print(f"\nHasil prediksi (untuk analisis kualitatif) berhasil disimpan ke: {prediction_metrics_csv_path}")
                print("\nUntuk analisis kegagalan model (error analysis), periksa file prediction_metrics.csv secara manual.")
            except Exception as e:
                print(f"ERROR: Gagal menyimpan prediction_metrics.csv: {e}")
        else:
            print("DataFrame metrik prediksi kosong.")
    else:
        print("Tidak ada data prediksi untuk disimpan.")

else:
    print("Tidak dapat melanjutkan eksekusi evaluasi. Pastikan data kasus, model/vektor, fungsi retrieve/predict_outcome, dan sample_queries tersedia.")

print("\nPROSES TAHAP 5 (MODEL EVALUATION) SELESAI.")

Folder '/content/drive/MyDrive/CBR_Data/data/eval' sudah ada di Google Drive.
Folder '/content/drive/MyDrive/CBR_Data/data/results' sudah ada di Google Drive.
Berhasil memuat 5 query dari queries.json.

--- Melakukan Retrieval dan Prediksi untuk Mengumpulkan Hasil Evaluasi ---
  Memproses Query ID: Q1_Narkotika_1

Melakukan retrieval untuk query: 'Seorang terdakwa ditangkap karena memiliki 5 gram ...' (top-k=5)
Retrieval selesai. Ditemukan 5 kasus.

Memprediksi solusi untuk query: 'Seorang terdakwa ditangkap karena memiliki 5 gram ...'

Melakukan retrieval untuk query: 'Seorang terdakwa ditangkap karena memiliki 5 gram ...' (top-k=5)
Retrieval selesai. Ditemukan 5 kasus.
DEBUG: Solusi prediksi diambil dari kasus dengan skor kemiripan tertinggi (0.2941).
Prediksi solusi selesai.
  Memproses Query ID: Q2_Narkotika_2

Melakukan retrieval untuk query: 'Kasus peredaran gelap narkotika jenis ganja yang m...' (top-k=5)
Retrieval selesai. Ditemukan 5 kasus.

Memprediksi solusi untuk query: 'Ka